In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train=pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
test=pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
submission=pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')

train.head()

In [ ]:
train[train['eeg_id'] != 1628180742].head()

In [ ]:
train.info()

In [ ]:
train.tail()

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
submission.head()

In [ ]:
def resumetable(df):
    print(f'Data shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Features'})
    summary['Number of Nan'] = df.isnull().sum().values
    summary['Number of Unique values'] = df.nunique().values
    summary['First value'] = df.loc[0].values
    summary['Second value'] = df.loc[1].values
    
    return summary
   
summary = resumetable(train)
summary

In [ ]:
t_copy = train.copy()
t_copy =t_copy.sort_values(['eeg_id'])
t_copy

In [ ]:
print([t_copy['eeg_id'].min(),t_copy['eeg_id'].max()])
print([t_copy['spectrogram_id'].min(),t_copy['spectrogram_id'].max()])
print([t_copy['patient_id'].min(),t_copy['patient_id'].max()])

In [ ]:
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt #시각화 툴
stats.pearsonr(train['eeg_id'],train['spectrogram_id'] )

In [ ]:
stats.pearsonr(t_copy['eeg_id'],t_copy['spectrogram_id'] )

In [ ]:
sns.histplot(x =t_copy['eeg_id'], y=t_copy['spectrogram_id'])

In [ ]:
sns.scatterplot(x =t_copy['eeg_id'], y=t_copy['spectrogram_id'])
plt.show()

In [ ]:
sns.pairplot(t_copy[['eeg_id','spectrogram_id','patient_id']])
plt.show

In [ ]:
t_copy = t_copy.drop_duplicates('eeg_id')
t_copy

In [ ]:
sns.histplot(x=t_copy['eeg_id'])

In [ ]:
sns.scatterplot(t_copy['eeg_id'])
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(15,5))

sns.boxplot(t_copy['eeg_id'],ax=ax[0])
sns.boxplot(t_copy['spectrogram_id'],ax=ax[1])
sns.boxplot(t_copy['patient_id'],ax=ax[2])

In [ ]:
sns.kdeplot(t_copy['eeg_id'])

In [ ]:
t_copy2 = train.copy()
t_copy2

In [ ]:
t_drop = t_copy2.drop_duplicates(['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote' ])
t_drop

In [ ]:
import missingno as msno
t_copy2 = train.copy()
msno.bar(df=t_copy2.iloc[:, :], figsize=(13, 6))

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def write_percent(ax, total_size):
    '''도형 객체를 순회하며 막대 그래프 상단에 타깃값 비율 표시'''
    for patch in ax.patches:
        height = patch.get_height()     # 도형 높이(데이터 개수)
        width = patch.get_width()       # 도형 너비
        left_coord = patch.get_x()      # 도형 왼쪽 테두리의 x축 위치
        percent = height/total_size*100 # 타깃값 비율
        
        # (x, y) 좌표에 텍스트 입력
        ax.text(left_coord + width/2.0,     # x축 위치
                height + total_size*0.001,  # y축 위치
                '{:1.3f}%'.format(percent), # 입력 텍스트
                ha='center')                # 가운데 정렬
    
mpl.rc('font', size=15)
plt.figure(figsize=(7, 6))
ax = sns.countplot(x='expert_consensus', data=train)
write_percent(ax, len(train)) # 비율 표시
ax.set_title('Target Distribution');

In [ ]:
def resumetable(df):
    print(f'Data shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Features'})
    summary['Number of Nan'] = df.isnull().sum().values
    summary['Number of Unique values'] = df.nunique().values
    summary['First value'] = df.loc[0].values
    summary['Second value'] = df.loc[1].values
    
    return summary
   
summary = resumetable(train)
summary

In [ ]:
import matplotlib.gridspec as gridspec

def plot_target_ratio_by_features(df, features, num_rows, num_cols, 
                                  size=(12, 18)):
    mpl.rc('font', size=9) 
    plt.figure(figsize=size)                     # 전체 Figure 크기 설정
    grid = gridspec.GridSpec(num_rows, num_cols) # 서브플롯 배치
    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 좌우/상하 여백 설정

    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        # ax축에 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
        sns.barplot(x=feature, y='expert_consensus', data=df, palette='Set2', ax=ax)

t_copy3 = train.drop('expert_consensus',axis=1)
bin_features = t_copy3.columns
# 이진 피처 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
plot_target_ratio_by_features(train, bin_features, 6, 3) # 6행 3열 배치

In [ ]:
plt.figure(figsize=(10, 8))
cont_corr = t_copy3[bin_features].corr()     # corr with every feature except target
sns.heatmap(cont_corr, annot=True, cmap='OrRd'); # 히트맵 그리기
plt.savefig('상관계수1.jpg', format='jpeg')

In [ ]:
feature = ['Seizure', 'GPD', 'LRDA', 'Other', 'GRDA', 'LPD']

t_copy4 = train.copy()

# Create a dictionary to map feature names to their index numbers
feature_index = {feat: idx for idx, feat in enumerate(feature)}

# Apply the mapping to create a new column 'transform'
t_copy4['transform'] = t_copy4['expert_consensus'].map(feature_index)

t_copy4


In [ ]:
plt.figure(figsize=(10, 8))
t_copy4 = t_copy4.drop('expert_consensus',axis=1)
col = t_copy4.columns
cont_corr = t_copy4[col].corr()     # corr with target(transform)
sns.heatmap(cont_corr, annot=True, cmap='OrRd'); # 히트맵 그리기
plt.savefig('상관계수2.jpg', format='jpeg')

In [ ]:
plt.figure(figsize=(10, 8))
col = t_copy4.columns
cont_corr = t_copy4[col].corr(method='spearman')     # 연속형 피처 간 상관관계 
sns.heatmap(cont_corr, annot=True, cmap='OrRd'); # 히트맵 그리기
plt.savefig('상관계수2.jpg', format='jpeg')

In [ ]:
def resumetable(df):
    print(f'데이터 세트 형상: {df.shape}') 
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입']) #데이터 타입별로 서머리
    summary = summary.reset_index() #그렇게 구한 서버리를 index 리셋
    summary = summary.rename(columns={'index': '피처'}) #피처를 
    summary['결측값 개수'] = df.isnull().sum().values #결측값 개수 열 추가
    summary['고윳값 개수'] = df.nunique().values #고윳값 개수 열 추가
    summary['첫 번째 값'] = df.loc[0].values #첫째값
    summary['두 번째 값'] = df.loc[1].values #둘째값
    
    return summary
   
resumetable(train)

In [ ]:
def resumetable(df):
    print(f'데이터 세트 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    
    summary['결측값 개수'] = df.isnull().sum().values #결측값 개수 열 추가
    summary['고윳값 개수'] = df.nunique().values
    summary['데이터 종류'] = None
    for col in df.columns: #데이터 종류 추가
        if 'id' in col:
            summary.loc[col, '데이터 종류'] = 'id형'
        elif df[col].dtype == object:
            summary.loc[col, '데이터 종류'] = '명목형'
        elif df[col].dtype == float:
            summary.loc[col, '데이터 종류'] = '연속형'
        elif 'vote' in col:
            summary.loc[col, '데이터 종류'] = '투표형'
    summary['첫 번째 값'] = df.loc[0].values #첫째값
    summary['두 번째 값'] = df.loc[1].values #둘째값
    return summary


summary=resumetable(train)
summary

In [ ]:
id_features = summary[summary['데이터 종류'] == 'id형'].index

def plot_target_ratio_by_features(df, features, num_rows, num_cols, 
                                  size=(12, 18)):
    mpl.rc('font', size=9) 
    plt.figure(figsize=size)                     # 전체 Figure 크기 설정
    grid = gridspec.GridSpec(num_rows, num_cols) # 서브플롯 배치
    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 좌우/상하 여백 설정

    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        # ax축에 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
        sns.barplot(x=feature, y='expert_consensus', data=df, palette='Set2', ax=ax)

t_copy5= train.copy()
print(id_features)
for idx, id_features in enumerate(id_features):
    # 값을 5개 구간으로 나누기
    t_copy5[id_features] = pd.cut(t_copy5[id_features], 5)

In [ ]:
from collections import Counter
target_votes = Counter(list(train['expert_consensus']))
target_votes = {f"{k.lower()}_vote":v for k,v in target_votes.items()}
total_votes = sum([v for _,v in target_votes.items()])
mean_vote_ratio = {k:(target_votes[k]/total_votes) for k,target in target_votes.items()}
print(target_votes)
print(total_votes)
print(mean_vote_ratio)